In [ ]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr

# Set style
sns.set(style="whitegrid")


In [ ]:
# path to local CSV file
file_path = 'Mushrooms.data.csv' 

# Column names as per the dataset provided
column_names = [
    'class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor',
    'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape',
    'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 
    'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 
    'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat'
]

# Load the dataset (with proper column names if the file doesn't include them)
df = pd.read_csv(file_path, sep=',', names=column_names, header=None)

# Display basic information about the dataset
df.info()

# Display the first few rows of the dataset
df.head()


In [ ]:
# Clean the 'class' column if necessary (standardizing 'poisonous' -> 'p' and 'edible' -> 'e')
df['class'] = df['class'].replace({'poisonous': 'p', 'edible': 'e'})

# Check the unique values in the 'class' column
print(df['class'].unique())

In [ ]:
# 1. **Class Distribution (Edible vs Poisonous)**
# Class distribution (edible vs poisonous)
class_distribution = df['class'].value_counts()

# Plotting the class distribution (Edible vs Poisonous)
plt.figure(figsize=(6, 4))
sns.countplot(x='class', data=df, hue='class', palette="Set2", legend=False)
plt.title('Class Distribution (Edible vs Poisonous)')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

# Display the class distribution count
print(class_distribution)

In [ ]:
# 2. **Check for Missing Values**
# Check if there are any missing values in the dataset
missing_values = df.isnull().sum()

# Display columns with missing values (if any)
print("Missing values per column:")
print(missing_values[missing_values > 0])

In [ ]:
# 3. **Feature Distributions**
# Plot the distribution of the first few categorical features (to understand their distribution)
plt.figure(figsize=(14, 10))
for i, column in enumerate(df.columns[1:5]):  # Plotting the first 4 columns
    plt.subplot(2, 2, i + 1)
    sns.countplot(x=column, data=df, hue='class', palette="Set1", legend=False)  # Updated line
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# 4. **Calculate Point-Biserial Correlation between Features and Target (Class)**
# Convert target ('class') to binary values (1 for poisonous, 0 for edible)
df['class'] = df['class'].map({'p': 1, 'e': 0})

# Calculate point-biserial correlation for each feature with the target variable
correlations = []
for column in df.columns[1:]:  # Skip the target column ('class')
    # Encode each categorical feature using one-hot encoding
    feature = pd.get_dummies(df[column], drop_first=True)  # One-hot encoding
    for col in feature.columns:
        # Calculate the point-biserial correlation between the target and feature
        corr, _ = pointbiserialr(df['class'], feature[col])  # Point-biserial correlation
        correlations.append((column, col, corr))

# Convert the list of correlations to a DataFrame for easier viewing
corr_df = pd.DataFrame(correlations, columns=['Feature', 'Encoded Feature', 'Correlation'])

# Sort correlations by absolute value of correlation to get the most important ones
corr_df = corr_df.sort_values(by='Correlation', ascending=False)

# Display the top correlated features
print("Top Correlations with Target (Poisonous Mushroom):")
print(corr_df.head())

In [ ]:
# 5. # Plotting the top 10 correlations with the target variable (Poisonous Mushroom)
top_corr_df = corr_df.head(10)  # Select top 10 most correlated features

plt.figure(figsize=(14, 8))  # Increase plot size for better readability
sns.barplot(x='Correlation', y='Encoded Feature', data=top_corr_df, hue='Encoded Feature', palette='Blues_d', dodge=False, errorbar=None, legend=False)
plt.title('Top 10 Point-Biserial Correlations with Target (Poisonous Mushroom)')
plt.xlabel('Correlation with Target')
plt.ylabel('Encoded Features')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
# 6. # Generate a heatmap for feature correlations (using one-hot encoded variables)
df_dummies = pd.get_dummies(df.drop('class', axis=1))

# Calculate the correlation matrix
corr_matrix = df_dummies.corr()

# Filter the correlation matrix to only include strong correlations (above a threshold)
threshold = 0.7
strong_corr_matrix = corr_matrix[corr_matrix.abs() > threshold]

# Set up the figure size for better readability
plt.figure(figsize=(16, 12))

# Generate the heatmap with annotations disabled and a clearer color scheme
sns.heatmap(strong_corr_matrix, cmap="coolwarm", annot=False, fmt=".2f", cbar_kws={'label': 'Correlation'}, linewidths=0.5)
plt.title('Filtered Correlation Heatmap of Categorical Features', fontsize=16)
plt.show()